In [2]:
import pandas as pd
import numpy as np
import csv
import pymorphy2

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from string import punctuation

import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

import math

In [3]:
import sklearn.metrics.pairwise as metrics
from scipy.spatial import distance

In [4]:
df = pd.read_csv('dataset.csv', encoding = 'utf8')
df.describe()

,tokens,name,text,author,cat,url
count,25026,25026,25026,25026,25026,25026
unique,25026,24391,25026,9045,20403,25026
top,этот книга десятилетие определять судьба чело...,Возвращение,"Эта книга о десятилетии, определяющем судьбу ч...",Неустановленный автор,Современные детективы;,https://mybook.ru/author/meg-dzhej/vazhnye-god...
freq,1,13,1,471,130,1


In [6]:
docs = np.array(df['tokens'].values)
tfidf_vectorizer = TfidfVectorizer()
values = tfidf_vectorizer.fit_transform(docs)

In [7]:
stop = stopwords.words('russian')
pymorph = pymorphy2.MorphAnalyzer()
def q_preprocess(query):
    query = query.replace(r"[^\w]", " ")
    query = query.replace(r"[\d]", " ")
    query = query.lower()
    filtered_sentence = ''
    query = nltk.word_tokenize(query)
    for w in query:
        if w not in stop:
            filtered_sentence = filtered_sentence + ' ' + pymorph.parse(w)[0].normal_form
    return filtered_sentence

def tfidf_query_evk(query, number=1, tfidf_vectorizer=tfidf_vectorizer):
    query = q_preprocess(query)
    vector_query = tfidf_vectorizer.transform([query])
    m = metrics.euclidean_distances(vector_query, values)
    buf_m = np.sort(m[0])[0:number]
    f = m[0].tolist()
    ind = [ f.index(i) for i in buf_m ]
    #вернет индексы максимумов по мере и значения, но не сортированные
    #ind = np.argpartition(m[0], -number)[-number:]
    #return ind, m[0][ind]
    return buf_m, ind, docs[ind] #  , data['ans'][ind].tolist()

def tfidf_query_cos(query, number=1, tfidf_vectorizer=tfidf_vectorizer):
    query = q_preprocess(query)
    vector_query = tfidf_vectorizer.transform([query])
    m = metrics.cosine_similarity(vector_query, values)
    buf_m = -np.sort(-m[0])[0:number]
    f = m[0].tolist()
    ind = [ f.index(i) for i in buf_m ]
    #вернет индексы максимумов по мере и значения, но не сортированные
    #ind = np.argpartition(m[0], -number)[-number:]
    #return ind, m[0][ind]
    return buf_m, ind, docs[ind], df['text'][ind].tolist() #  df.iloc[ind]['name']

In [8]:
%%time
tfidf_query_cos(df['tokens'][10111], 1)

Wall time: 62.3 ms


(array([1.]),
 [10111],
 array([' чтение сказка легенда доставлять удовольствие ребёнок взрослый книга войти красивый английский легенда биннорь принцесса кентербрать знаменитый беовульф др текст подготовить уровень продолжать учить английский язык нижний ступень pre intermediate снабдить комментарий легенда предлагаться упражнение проверка понимание текст конец книга ангнуть русский словарик издание рассчитать стремиться читать английский язык'],
       dtype=object),
 ['Чтение сказок и легенд доставляет удовольствие не только детям, но и взрослым.В книгу вошли красивые английские легенды «Биннори», «Принцесса Кентербери», знаменитый «Беовульф» и др.Тексты подготовлены для уровня 2 (для продолжающих учить английский язык нижней ступени Pre-Intermediate) и снабжены комментариями.К легендам предлагаются упражнения для проверки понимания текста, а в конце книги – англо-русский словарик.Издание рассчитано на всех, кто стремится читать на английском языке.'])

In [43]:
def jaccard_similarity(list1, list2):
    list1 = list1.split()
    list2 = list2.split()
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def tfidf_query_jac(query, number=1):
    query = q_preprocess(query)
    jac = []
    for i in range (0, len(docs)):
        j = jaccard_similarity(query, docs[i])
        if (j > 0):
            jac.append([j, i, docs[i]])
    jac.sort(reverse = True)
    return jac[0:number]  #buf_m, ind, docs[ind] #  , data['ans'][ind].tolist()

In [52]:
tfidf_query_jac('два человека в ссср ищут стулья в которых спрятаны сокровища', 3)

[[0.1111111111111111,
  24850,
  ' сборник собрать китайский притча который хранить мудрость тысячелетие помогать постичь сущность природа человек'],
 [0.1,
  9951,
  ' данный издание войти такой знаменитый произведение виткора гюго собор парижский богоматерь человек который смеяться гаврош'],
 [0.09523809523809523,
  18148,
  ' сборник комментировать закон который образовать фундамент лидерство любой область признать повесть человек себя широкий круг читатель']]

### Тестирование

In [7]:
def otiai_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    sqrt = math.sqrt((len(list1) * len(list2)))
    return float(intersection) / sqrt

In [26]:
tfidf_query_cos(df['tokens'][4554], 2)

(array([0.99901902, 0.19806065]),
 [4554, 3881],
 array([' принять боевой крещение июль год могилёв дивизия пойти бой против сам сильный соединение вермахт й танковый группа гудериана иметь строй тысяча штык месяц третий счёт окружение прорваться боец изранить обескровить чёрный голод недосып едва держаться нога видеть круг фронтовой ад несломленный непобеждённый непобедимый роман основать реальный событие автор опросить остаться живой ветеран й стрелковый дивизия пройти пешком весь боевой путь книга слово вымысел умолчание парадное миф это чистый неразбавленный обжигать фронтовой грамм перехватывать дух пробирать слеза окопный правда великий отечественный это гимн непобедимый русский пехота стоять насмерть лето год',
        ' провалиться xxi век год пройти круг фронтовой ад прорываться окружение трофейный танк воевать пехота фронтовой разведка танковый бригада который сгорать дотла передовой считать день собственный опыт убедиться насколько беспощадный неповоротливый история изменить

In [122]:
jcds = np.array([])
for i in range(100):
    sims = tfidf_query_cos(df['tokens'][i], 2)
    if sims[0][1] > 0.8:
        print(str(i) + ' ' + df['text'][i])
        print(df['text'][sims[1][0]])
        print('-------------------------------')
        jd = otiai_similarity(df['cat'][i].split(';'), df['cat'][sims[1][1]].split(';'))
        jcds = np.append(jcds, jd)
print(sum(jcds)/len(jcds))
print(len(jcds))

40 Одна из лучших фэнтези-саг за всю историю существования жанра.Оригинальное, масштабное эпическое произведение, одновременно и свободное от влияния извне, и связанное с классической мифологической, легендарной и сказовой традицией.Шедевр не только писательского мастерства Анджея Сапковского, но и переводческого искусства Евгения Павловича Вайсброта.«Сага о Геральте» – в одном томе.Бесценный подарок и для поклонника прекрасной фантастики, и для ценителя просто хорошей литературы.Перед читателем буквально оживает необычный, прекрасный и жестокий мир литературной легенда, в котором обитают эльфы и гномы, оборотни, вампиры и «низушки»-хоббиты, драконы и монстры, – но прежде всего люди.Очень близкие нам, понятные и человечные люди – такие как мастер меча ведьмак Геральт, его друг, беспутный менестрель Лютик, его возлюбленная, прекрасная чародейка Йеннифэр, и приемная дочь – безрассудно отважная юная Цири…Больше интересных фактов об этой книге и втором сезоне сериала читайте в ЛитРес: Журн

447 Перед вами книга из серии «Классика в школе», в которой собраны все произведения, изучающиеся в начальной школе, средних и старших классах. Не тратьте время на поиски литературных произведений, ведь в этих книгах есть все, что необходимо прочесть по школьной программе: и для чтения в классе, и для внеклассных заданий. Избавьте своего ребенка от длительных поисков и невыполненных уроков.В книгу включены рассказы «Алешкино сердце», «Нахаленок», «Наука ненависти», «Судьба человека» и публицистические статьи, которые изучают в 5, 6, 7, 9 и 11-м классах.
Перед вами книга из серии «Классика в школе», в которой собраны все произведения, изучающиеся в начальной школе, средних и старших классах. Не тратьте время на поиски литературных произведений, ведь в этих книгах есть все, что необходимо прочесть по школьной программе: и для чтения в классе, и для внеклассных заданий. Избавьте своего ребенка от длительных поисков и невыполненных уроков.В книгу включены рассказы «Алешкино сердце», «Нахал